In [1]:
!nvidia-smi

Wed Apr 17 06:30:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:37:00.0 Off |                    0 |
| N/A   70C    P0             47W /  250W |    1315MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:2.2.2
cuda version: 12.1
cudnn version:8902


In [18]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import urllib.request
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from torch import nn
from tqdm import tqdm

In [10]:
from transformers import AutoTokenizer, BartForConditionalGeneration

checkpoint_name = 'gogamza/kobart-summarization'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = BartForConditionalGeneration.from_pretrained(checkpoint_name)

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


vocab.json:   0%|          | 0.00/446k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [11]:
# Load the dataset
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
file_path = 'ChatBotData.csv'

In [12]:
data = pd.read_csv('ChatBotData.csv',encoding='utf-8-sig')

In [13]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [14]:
class ChatbotDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['Q']
        response_text = self.data.iloc[idx]['A']
        
        # Tokenize the input and response text
        input_tokens = self.tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=128)
        response_tokens = self.tokenizer.encode(response_text, truncation=True, padding='max_length', max_length=128)
        
        return {'input_ids': input_tokens, 'attention_mask': [1] * len(input_tokens), 'decoder_input_ids': response_tokens[:-1], 'decoder_attention_mask': [1] * len(response_tokens[:-1]), 'labels': response_tokens[1:]}

In [15]:
train_dataset = ChatbotDataset('ChatBotData.csv', tokenizer)
data_collator = lambda data: {'input_ids': [item['input_ids'] for item in data], 
                              'attention_mask': [item['attention_mask'] for item in data],
                              'decoder_input_ids': [item['decoder_input_ids'] for item in data],
                              'decoder_attention_mask': [item['decoder_attention_mask'] for item in data],
                              'labels': [item['labels'] for item in data]}

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)

In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [17]:
optimizer = AdamW(model.parameters(), lr=1e-5)
model.to(device)
model.train()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [19]:
criterion = nn.CrossEntropyLoss(ignore_index=model.config.pad_token_id)

In [20]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        decoder_input_ids = torch.tensor(batch['decoder_input_ids']).to(device)
        decoder_attention_mask = torch.tensor(batch['decoder_attention_mask']).to(device)
        labels = torch.tensor(batch['labels']).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask)
        logits = outputs.logits
        loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch + 1} loss: {epoch_loss / len(train_dataloader):.5f}')

100%|██████████| 1478/1478 [01:53<00:00, 13.02it/s]


Epoch 1 loss: 4.55159


100%|██████████| 1478/1478 [01:49<00:00, 13.55it/s]


Epoch 2 loss: 3.79484


100%|██████████| 1478/1478 [01:48<00:00, 13.59it/s]


...

Epoch 8 loss: 1.49610


100%|██████████| 1478/1478 [02:15<00:00, 10.88it/s]


Epoch 9 loss: 1.21438


100%|██████████| 1478/1478 [02:15<00:00, 10.89it/s]

Epoch 10 loss: 0.98194


In [21]:
torch.save(model.state_dict(), 'bart_chatbot.pt')

In [22]:
model.eval()

def conversations():
  input_text = input('User: ')
  input_tokens = tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=128)
  input_ids = torch.tensor([input_tokens]).to(device)
  output_tokens = model.generate(input_ids, max_length=10, num_beams=4, early_stopping=True)
  output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  print(f'ChatBot: {output_text}')

In [23]:
conversations()

ChatBot: 녕녕녕녕녕녕녕녕


In [24]:
conversations()

ChatBot: 너너 이름이 뭐 뭐 뭐 뭐 뭐


In [25]:
conversations()

ChatBot: 말을 많이말을 많이 많이 많이 많이 많이
